# A Puzzle with Unintended Sharing in Python Lists

In [ ]:
n = 5
ls = [[]] * n
for i in range(5):
    ls[i].append(i)
ls[0]

# An Interlude on Estimating Running Time of Python Programs

In [ ]:
l = []
# Make a new empty list.  (Initializing with multiple elements wouldn't count as primitive anymore!)
l.append(5)
l.append(7)
# Appending to a list.
v = l[1]
# Reading from a position.
l[0] = 6
# Writing to a position.

An example with lines that use multiple primitive operations:

In [ ]:
l1 = list(range(10))
l2 = l1[1:10]
l2

It really acts like this:

In [ ]:
l2 = []
for i in range(1, 10):
    l2.append(l1[i])
l2

Another costly example:

In [ ]:
l1[:5] + ['hi'] + l1[5:]

What that code *really* does:

In [ ]:
l2 = []
for i in range(5):
    l2.append(l1[i])
l2.append('hi')
for i in range(5, 10):
    l2.append(l1[i])
l2

# Linked Lists

In [ ]:
# Basic construction of linked lists: create an empty one.
def nil():
    return None

# Create a new list that starts with x and then continues as list ls.
def cons(x, ls):
    return {"data": x,
            "next": ls}
    # Note that we are using Python dictionaries here to represent our "clusters in memory,"
    # but we will switch to a nicer implementation next.

# Is a list empty?
def isEmpty(ls):
    return ls == None

# Return the first element of a nonempty list (raises exception if empty).
def head(ls):
    return ls["data"]

# Return the part of a list after the first element (raises exception if empty).
def tail(ls):
    return ls["next"]

In [ ]:
l = cons('A', cons('B', cons('C', nil())))
print(isEmpty(l))
print(head(l))
print(head(tail(l)))
print(head(tail(tail(l))))
print(isEmpty(tail(tail(tail(l)))))

In [ ]:
def length(ls):
    len = 0

    while not isEmpty(ls):
        len += 1
        ls = tail(ls)

    return len

length(l)

In [ ]:
def rev(ls):
    ret = nil()

    while not isEmpty(ls):
        ret = cons(head(ls), ret)
        ls = tail(ls)

    return ret

print(head(l))
l_reversed = rev(l)
print(head(l_reversed))
print(head(l))

In [ ]:
def concat(ls1, ls2):
    if isEmpty(ls1):
        return ls2
    else:
        return cons(head(ls1), concat(tail(ls1), ls2))

lc = concat(l, l_reversed)
[head(lc), head(tail(lc)), head(tail(tail(lc))), head(tail(tail(tail(lc)))), head(tail(tail(tail(tail(lc)))))]

In [ ]:
def upto(n):
    ls = nil()

    for i in range(n):
        ls = cons(i, ls)

    return ls

[head(upto(3)), head(tail(upto(3))), head(tail(tail(upto(3)))), isEmpty(tail(tail(tail(upto(3)))))]

In [ ]:
def test_lists():
    ls = cons(1, cons(2, cons(3, nil())))
    assert not isEmpty(ls)
    assert head(ls) == 1
    assert head(tail(ls)) == 2
    assert head(tail(tail(ls))) == 3
    assert isEmpty(tail(tail(tail(ls))))

    assert length(ls) == 3
    assert length(tail(ls)) == 2

    assert rev(ls) == cons(3, cons(2, cons(1, nil())))

    assert concat(cons(4, cons(5, nil())), ls) == cons(4, cons(5, cons(1, cons (2, cons (3, nil())))))

test_lists()

# Object-Oriented Linked Lists

In [ ]:
class List:
    def concat(self, other):
        return self.rev().rev(other)

In [ ]:
class Nil(List):
    def toArray(self):
        return []

    def isEmpty(self):
        return True

    def head(self):
        raise ValueError("empty list doesn't have a head")

    def tail(self):
        raise ValueError("empty list doesn't have a tail")

    def length(self):
        return 0

    def rev(self, acc=None):
        if acc == None:
            return self
        else:
            return acc

In [ ]:
class Cons(List):
    def __init__(self, hd, tl):
        self.hd = hd
        self.tl = tl

    def toArray(self):
        return [self.hd] + self.tl.toArray()

    def isEmpty(self):
        return False

    def head(self):
        return self.hd

    def tail(self):
        return self.tl

    def length(self):
        raise NotImplementedError

    def rev(self, acc=Nil()):
        raise NotImplementedError

In [ ]:
def test_oo_lists():
    ls = Cons(1, Cons(2, Cons(3, Nil())))
    assert not ls.isEmpty()
    assert ls.head() == 1
    assert ls.tail().head() == 2
    assert ls.tail().tail().head() == 3
    assert ls.tail().tail().tail().isEmpty()

    assert ls.length() == 3
    assert ls.tail().length() == 2

    assert ls.rev().toArray() == [3, 2, 1]

    assert Cons(4, Cons(5, Nil())).concat(ls).toArray() == [4, 5, 1, 2, 3]

test_oo_lists()

# Representing an Archive of Course Schedules

In [ ]:
class Subject:
    def __init__(self, num, titl):
        self.num = num
        self.titl = titl

    def number(self):
        return self.num

    def title(self):
        return self.titl

Here's one semester's schedule as an array.

In [ ]:
initial_schedule = [Subject(100, "Introduction to N-Queens"),
                    Subject(101, "Insights in Image Processing"),
                    Subject(110, "Let's Read the Dictionary"),
                    Subject(111, "Deconstructing Kevin Bacon"),
                    Subject(120, "Readings in Recursion"),
                    Subject(121, "Advanced Minesweeper"),
                    Subject(130, "Sudoku Studies"),
                    Subject(131, "A Brief History of Camping"),
                    Subject(200, "Linked Lists Lab"),
                    Subject(201, "Autocomplete Seminar")]

# A Simple Implementation with Arrays

In [ ]:
class ArraySchedule:
    def __init__(self):
        self.subjects = []

    def lookup(self, subject_number):
        """Return the title of subject in schedule, or None if not found."""

        for subject in self.subjects:
            if subject.number() == subject_number:
                return subject.title()
        return None

    def add(self, subject):
        """Return a new schedule, based on the given one, but with a new entry
        added with given subject and title."""

        new = ArraySchedule()
        new.subjects = self.subjects + [subject]
        return new

# Neat trick: we can write tests generically in a class like this one!
def tests_helper(sched):
    assert sched.lookup(101) == "Insights in Image Processing"

    latest_and_greatest = sched.add(Subject(102, "Tenets of Tent Packing"))
    assert latest_and_greatest.lookup(101) == "Insights in Image Processing"
    assert latest_and_greatest.lookup(102) == "Tenets of Tent Packing"
    assert sched.lookup(102) == None
    # Important: the old schedule is still around and does not contain the new course!

def tests(sched):
    for subject in initial_schedule:
        sched = sched.add(subject)

    tests_helper(sched)

tests(ArraySchedule())

## Using Linked Lists

In [ ]:
class NilSchedule:
    def add(self, subject):
        return ConsSchedule(subject, self)

    def lookup(self, subject_number):
        return None

class ConsSchedule:
    def __init__(self, subject, rest):
        self.hd = subject
        self.tl = rest

    def add(self, subject):
        return ConsSchedule(subject, self)

    def lookup(self, subject_number):
        if self.hd.number() == subject_number:
            return self.hd.title()
        else:
            return self.tl.lookup(subject_number)

tests(NilSchedule())

## Sorted Linked Lists

In [ ]:
class NilSortedSchedule:
    def add(self, subject):
        return ConsSortedSchedule(subject, self)

    def lookup(self, subject_number):
        return None

class ConsSortedSchedule:
    def __init__(self, subject, rest):
        self.hd = subject
        self.tl = rest

    def add(self, subject):
        return ConsSortedSchedule(subject, self)

    def lookup(self, subject_number):
        if self.hd.number() == subject_number:
            return self.hd.title()
        else:
            return self.tl.lookup(subject_number)

tests(NilSortedSchedule())

## Binary Search Trees (a true classic data structure)

In [ ]:
class EmptyTreeSchedule:
    def add(self, subject):
        return NonemptyTreeSchedule(subject,
                                    EmptyTreeSchedule(),
                                    EmptyTreeSchedule())

    def lookup(self, subject_number):
        return None

class NonemptyTreeSchedule:
    def __init__(self, subject, leftChild, rightChild):
        self.subject = subject
        self.leftChild = leftChild
        self.rightChild = rightChild

    def add(self, subject):
        raise NotImplementedError

    def lookup(self, subject_number):
        if subject_number == self.subject.number():
            return self.subject.title()
        elif subject_number < self.subject.number():
            return self.leftChild.lookup(subject_number)
        else:
            return self.rightChild.lookup(subject_number)

tests(EmptyTreeSchedule())

### A Performance Gotcha

In [ ]:
import sys
sys.setrecursionlimit(10000)

tree = EmptyTreeSchedule()
for i in range(3000):
    tree = tree.add(Subject(i, "Redundancy"))
tree.lookup(2500)

# Sudoku Revisited

Here's a literal copy of the most optimized Sudoku code from the previous lecture.

In [ ]:
backtracks = 0

#x varies from entry1 to entry2 - 1, y varies from entry3 to entry4 - 1 
sectors = [ [0, 3, 0, 3], [3, 6, 0, 3], [6, 9, 0, 3],
            [0, 3, 3, 6], [3, 6, 3, 6], [6, 9, 3, 6],
            [0, 3, 6, 9], [3, 6, 6, 9], [6, 9, 6, 9] ]

#This procedure finds the next empty square to fill on the Sudoku grid
def findNextCellToFill(grid):
    #Look for an unfilled grid location
    for x in range(0, 9):
        for y in range(0, 9):
            if grid[x][y] == 0:
                return x,y
    return -1,-1

#This procedure checks if setting the (i, j) square to e is valid
def isValid(grid, i, j, e):
    rowOk = all([e != grid[i][x] for x in range(9)])
    if rowOk:
        columnOk = all([e != grid[x][j] for x in range(9)])
        if columnOk:
            #finding the top left x,y co-ordinates of
            #the section or sub-grid containing the i,j cell
            secTopX, secTopY = 3 *(i//3), 3 *(j//3)
            for x in range(secTopX, secTopX+3):
                for y in range(secTopY, secTopY+3):
                    if grid[x][y] == e:
                        return False
            return True
    return False

#This procedure makes implications based on existing numbers on squares
def makeImplications(grid, i, j, e):

    global sectors

    grid[i][j] = e
    impl = [(i, j, e)]

    done = False

    #Keep going till you stop finding implications
    while not done:
        done = True

        for k in range(len(sectors)):

            sectinfo = []

            #find missing elements in ith sector
            vset = {1, 2, 3, 4, 5, 6, 7, 8, 9}
            for x in range(sectors[k][0], sectors[k][1]):
                for y in range(sectors[k][2], sectors[k][3]):
                    if grid[x][y] != 0:
                        vset.remove(grid[x][y])

            #attach copy of vset to each missing square in ith sector
            for x in range(sectors[k][0], sectors[k][1]):
                for y in range(sectors[k][2], sectors[k][3]):
                    if grid[x][y] == 0:
                        sectinfo.append([x, y, vset.copy()])
            
            for m in range(len(sectinfo)):
                sin = sectinfo[m]
                
                #find the set of elements on the row corresponding to m and remove them
                rowv = set()
                for y in range(9):
                    rowv.add(grid[sin[0]][y])
                left = sin[2].difference(rowv)
                
                #find the set of elements on the column corresponding to m and remove them
                colv = set()
                for x in range(9):
                    colv.add(grid[x][sin[1]])
                left = left.difference(colv)
                             
                #check if the vset is a singleton
                if len(left) == 1:
                    val = left.pop()
                    if isValid(grid, sin[0], sin[1], val):
                        grid[sin[0]][sin[1]] = val
                        impl.append((sin[0], sin[1], val))
                        done = False
                
    return impl

#This procedure undoes all the implications
def undoImplications(grid, impl):
    for i in range(len(impl)):
        grid[impl[i][0]][impl[i][1]] = 0
    return

#This procedure fills in the missing squares of a Sudoku puzzle
#obeying the Sudoku rules by guessing when it has to and performing
#implications when it can
def solveSudokuMoreOpt(grid, i=0, j=0):

    global backtracks

    #find the next empty cell to fill
    i, j = findNextCellToFill(grid)
    if i == -1:
        return True

    for e in range(1, 10):
        #Try different values in i, j location
        if isValid(grid, i, j, e):

            impl = makeImplications(grid, i, j, e)
            
            if solveSudokuMoreOpt(grid, i, j):
                return True
            #Undo the current cell for backtracking
            backtracks += 1
            undoImplications(grid, impl)

    return False

def printSudoku(grid):
    numrow = 0
    for row in grid:
        if numrow % 3 == 0 and numrow != 0:
            print (' ')
        print (row[0:3], ' ', row[3:6], ' ', row[6:9])
        numrow += 1       
    return

diff  = [[0,0,5,3,0,0,0,0,0],
         [8,0,0,0,0,0,0,2,0],
         [0,7,0,0,1,0,5,0,0],
         [4,0,0,0,0,5,3,0,0],
         [0,1,0,0,7,0,0,0,6],
         [0,0,3,2,0,0,0,8,0],
         [0,6,0,5,0,0,0,0,9],
         [0,0,4,0,0,0,0,3,0],
         [0,0,0,0,0,9,7,0,0]]

backtracks = 0
solveSudokuMoreOpt(diff)
print ('Backtracks =', backtracks)

Remember all the work we did to track changes made to the board, so we could undo them when backtracking?  How about if we represent boards as *binary search trees* whose keys are coordinates?  Then we can maintain multiple boards at once, with significant sharing across them!  We'll do it in a slightly gross way, reusing our subject-scheduling trees, representing each board square as a subject whose number is its coordinates and name is the value of the square (e.g., a digit).

Here's the code again following that strategy, where comments like `# CHANGE! --->` indicate where we made modifications.

In [ ]:
#This procedure finds the next empty square to fill on the Sudoku grid
def findNextCellToFill(grid):
    #Look for an unfilled grid location
    for x in range(0, 9):
        for y in range(0, 9):
# CHANGE! --->
# Look up grid cell with a method, by coordinates.
            if grid.lookup((x, y)) == 0:
                return x,y
    return -1,-1

#This procedure checks if setting the (i, j) square to e is valid
def isValid(grid, i, j, e):
# CHANGE! --->
# Look up by coordinates.
    rowOk = all([e != grid.lookup((i, x)) for x in range(9)])
    if rowOk:
        columnOk = all([e != grid.lookup((x, j)) for x in range(9)])
        if columnOk:
            #finding the top left x,y co-ordinates of
            #the section or sub-grid containing the i,j cell
            secTopX, secTopY = 3 *(i//3), 3 *(j//3)
            for x in range(secTopX, secTopX+3):
                for y in range(secTopY, secTopY+3):
                    if grid.lookup((x, y)) == e:
                        return False
            return True
    return False

#This procedure makes implications based on existing numbers on squares
def makeImplications(grid, i, j, e):

    global sectors

# CHANGE! --->
# Generate a _new_ grid without overwriting the old one.
# (For simplicity, we'll use the same variable to hold the new one,
# since we never need to reference the old version again in this function.)
    grid = grid.add(Subject((i, j), e))
# CHANGE! --->
# We no longer track a list of implications.
#   impl = [(i, j, e)]

    done = False

    #Keep going till you stop finding implications
    while not done:
        done = True

        for k in range(len(sectors)):

            sectinfo = []

            #find missing elements in ith sector
            vset = {1, 2, 3, 4, 5, 6, 7, 8, 9}
            for x in range(sectors[k][0], sectors[k][1]):
                for y in range(sectors[k][2], sectors[k][3]):
# CHANGE! --->
# Look up by coordinates.
                    if grid.lookup((x, y)) != 0:
                        vset.remove(grid.lookup((x, y)))

            #attach copy of vset to each missing square in ith sector
            for x in range(sectors[k][0], sectors[k][1]):
                for y in range(sectors[k][2], sectors[k][3]):
# CHANGE! --->
# Look up by coordinates.
                    if grid.lookup((x,y)) == 0:
                        sectinfo.append([x, y, vset.copy()])
            
            for m in range(len(sectinfo)):
                sin = sectinfo[m]
                
                #find the set of elements on the row corresponding to m and remove them
                rowv = set()
                for y in range(9):
# CHANGE! --->
# Look up by coordinates.
                    rowv.add(grid.lookup((sin[0], y)))
                left = sin[2].difference(rowv)
                
                #find the set of elements on the column corresponding to m and remove them
                colv = set()
                for x in range(9):
# CHANGE! --->
# Look up by coordinates.
                    colv.add(grid.lookup((x, sin[1])))
                left = left.difference(colv)
                             
                #check if the vset is a singleton
                if len(left) == 1:
                    val = left.pop()
                    if isValid(grid, sin[0], sin[1], val):
# CHANGE! --->
# Set by coordinates, overwriting grid variable.
                        grid = grid.add(Subject((sin[0], sin[1]), val))
# CHANGE! --->
# Not maintaining impl anymore.
#                       impl.append((sin[0], sin[1], val))
                        done = False

# CHANGE! --->
# Return new grid, rather than implications.
    return grid

# CHANGE! --->
# We don't need an undoing procedure anymore!
#This procedure undoes all the implications
#def undoImplications(grid, impl):
#    for i in range(len(impl)):
#        grid[impl[i][0]][impl[i][1]] = 0
#    return

# CHANGE! --->
# Converting into our funky board format.
def convert_board(grid, empty):
    new_grid = empty
    for i in range(9):
        for j in range(9):
            new_grid = new_grid.add(Subject((i, j), grid[i][j]))
    return new_grid

#This procedure fills in the missing squares of a Sudoku puzzle
#obeying the Sudoku rules by guessing when it has to and performing
#implications when it can
def solveSudokuMoreOpt(grid, i=0, j=0):

    global backtracks
    
    #find the next empty cell to fill
    i, j = findNextCellToFill(grid)
    if i == -1:
        return True

    for e in range(1, 10):
        #Try different values in i, j location
        if isValid(grid, i, j, e):
# CHANGE! --->
# Save new grid in a separate variable here.
            new_grid = makeImplications(grid, i, j, e)
            
            if solveSudokuMoreOpt(new_grid, i, j):
                return True
            #Undo the current cell for backtracking
            backtracks += 1
# CHANGE! --->
# No undoing required; just go back to using the old value!
#           undoImplications(grid, impl)

    return False

diff  = [[0,0,5,3,0,0,0,0,0],
         [8,0,0,0,0,0,0,2,0],
         [0,7,0,0,1,0,5,0,0],
         [4,0,0,0,0,5,3,0,0],
         [0,1,0,0,7,0,0,0,6],
         [0,0,3,2,0,0,0,8,0],
         [0,6,0,5,0,0,0,0,9],
         [0,0,4,0,0,0,0,3,0],
         [0,0,0,0,0,9,7,0,0]]

backtracks = 0
solveSudokuMoreOpt(convert_board(diff, EmptyTreeSchedule()))
print ('Backtracks =', backtracks)

Notice that this version is slower than the undoing version, but one could argue we saved in code understandability.